# Just doing some neuroblastoma-specific stuff.

In [1]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from src.CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
# from Bio import AlignIO

In [2]:
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
samplesetname = "NB_to_run"

In [3]:
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

## create sample set for NB lines to run

In [4]:
# get list of all samples in the Sanger set
refwm.get_sample_sets()["samples"]["Sanger"]

['sanger_ACH-002089',
 'sanger_ACH-000956',
 'sanger_ACH-000948',
 'sanger_ACH-000323',
 'sanger_ACH-001002',
 'sanger_ACH-000905',
 'sanger_ACH-000973',
 'sanger_ACH-000896',
 'sanger_ACH-000070',
 'sanger_ACH-000411',
 'sanger_ACH-000649',
 'sanger_ACH-001306',
 'sanger_ACH-001307',
 'sanger_ACH-000137',
 'sanger_ACH-000008',
 'sanger_ACH-000558',
 'sanger_ACH-000201',
 'sanger_ACH-000788',
 'sanger_ACH-000740',
 'sanger_ACH-000657',
 'sanger_ACH-000219',
 'sanger_ACH-001442',
 'sanger_ACH-000697',
 'sanger_ACH-000757',
 'sanger_ACH-001328',
 'sanger_ACH-000555',
 'sanger_ACH-000157',
 'sanger_ACH-000681',
 'sanger_ACH-000052',
 'sanger_ACH-000429',
 'sanger_ACH-000528',
 'sanger_ACH-000046',
 'sanger_ACH-002389',
 'sanger_ACH-000880',
 'sanger_ACH-002209',
 'sanger_ACH-000105',
 'sanger_ACH-000269',
 'sanger_ACH-000838',
 'sanger_ACH-000940',
 'sanger_ACH-002210',
 'sanger_ACH-001443',
 'sanger_ACH-000222',
 'sanger_ACH-002058',
 'sanger_ACH-000248',
 'sanger_ACH-002211',
 'sanger_A

In [5]:
# get subset that haven't been run through the pipeline yet
# need to grab just these: 
nb_lines_to_run = pd.read_csv("../NB-copy-loss/data/NB_lines_to_run.csv") # I used an R script to get this list.
nb_lines_to_run = nb_lines_to_run["x"].to_list()
nb_lines_to_run

['sanger_ACH-000078',
 'sanger_ACH-000099',
 'sanger_ACH-000120',
 'sanger_ACH-000136',
 'sanger_ACH-000149',
 'sanger_ACH-000227',
 'sanger_ACH-000259',
 'sanger_ACH-000260',
 'sanger_ACH-000341',
 'sanger_ACH-000345',
 'sanger_ACH-000366',
 'sanger_ACH-000804',
 'sanger_ACH-001338',
 'sanger_ACH-001344',
 'sanger_ACH-001355',
 'sanger_ACH-001603',
 'sanger_ACH-001674',
 'sanger_ACH-001716',
 'sanger_ACH-002083',
 'sanger_ACH-002217',
 'sanger_ACH-002248',
 'sanger_ACH-002261',
 'sanger_ACH-002277',
 'sanger_ACH-002278',
 'sanger_ACH-002279',
 'sanger_ACH-002280',
 'sanger_ACH-002281',
 'sanger_ACH-002282',
 'sanger_ACH-002283',
 'sanger_ACH-002284',
 'sanger_ACH-002285',
 'sanger_ACH-002340',
 'sanger_ACH-002389']

In [6]:
# create new sample set in Terra for these lines
refwm.update_sample_set(samplesetname, nb_lines_to_run)

Sample set "NB_to_run" (33 samples) successfully updated.


In [7]:
# run the CGA characterization pipeline: will give MAF and ABSOLUTE as output
CGAmutations = refwm.get_config("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019")
CGAmutations

{'deleted': False,
 'inputs': {'CGA_Production_Analysis_Workflow.CrossCheckLaneFingerprints_Task.HaplotypeDBForCrossCheck': 'workspace.HAP_DB_FOR_CC',
  'CGA_Production_Analysis_Workflow.gatk_acnv_only.one_thousand_genomes_common_variants_minor_allele_freq_five': 'workspace.COMMON_VARIANTS_1000_GENOMES',
  'CGA_Production_Analysis_Workflow.VEP_Task.GNOMAD_FILE_IDX': 'workspace.GNOMAD_IDX',
  'CGA_Production_Analysis_Workflow.lego_plotter_task.mut_categs': 'workspace.MUT_CATEGORIES',
  'CGA_Production_Analysis_Workflow.MutectFC_Task.mutectIntervals': 'workspace.MUTECT_FC_INTERVALS',
  'CGA_Production_Analysis_Workflow.hasPicardMetrics_normal': '',
  'CGA_Production_Analysis_Workflow.DB_SNP_VCF_IDX': 'workspace.DB_SNP_VCF_IDX',
  'CGA_Production_Analysis_Workflow.ctrlName': '"RCRF9001_KIDNEY_Normal"',
  'CGA_Production_Analysis_Workflow.targetIntervals': 'workspace.ILLUMINA_TARGET_INTERVALS',
  'CGA_Production_Analysis_Workflow.readGroupBlackList': 'workspace.READ_GROUP_BLACKLIST',
  'CG

In [10]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

Successfully updated configuration getzlab/CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019
Successfully created submission 5f1e1d9c-b8cb-4263-bcf3-0c9c48d49f83.


In [11]:
terra.waitForSubmission(refwm, submission_id)

sanger_ACH-000078 for 0 jobs in submission 0. 2 mn elapsed.
sanger_ACH-000099
sanger_ACH-000120
sanger_ACH-000136
sanger_ACH-000149
sanger_ACH-000227
sanger_ACH-000259
sanger_ACH-000260
sanger_ACH-000341
sanger_ACH-000345
sanger_ACH-000366
sanger_ACH-000804
sanger_ACH-001338
sanger_ACH-001344
sanger_ACH-001355
sanger_ACH-001603
sanger_ACH-001674
sanger_ACH-001716
sanger_ACH-002083
sanger_ACH-002217
sanger_ACH-002248
sanger_ACH-002261
sanger_ACH-002277
sanger_ACH-002278
sanger_ACH-002279
sanger_ACH-002280
sanger_ACH-002281
sanger_ACH-002282
sanger_ACH-002283
sanger_ACH-002284
sanger_ACH-002285
sanger_ACH-002340
sanger_ACH-002389
0.0 of jobs Succeeded in submission 0.


TypeError: exceptions must derive from BaseException